## RNN pytorch

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

batch_size = len(sentences)
# n_step = 2
n_hidden = 5

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)

    return torch.LongTensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_batch(sentences)

print(word_dict)
print(input_batch)
print(target_batch)


{'like': 0, 'milk': 1, 'cat': 2, 'dog': 3, 'hate': 4, 'love': 5, 'coffee': 6, 'you': 7, 'i': 8}
tensor([[8, 0],
        [8, 5],
        [8, 4],
        [7, 0],
        [7, 5],
        [7, 4]])
tensor([3, 6, 1, 2, 1, 6])


9

In [10]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.embedding = nn.Embedding(n_class, n_class)  # Using embedding layer
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, batch_first=True)
        self.fc = nn.Linear(n_hidden, n_class)  # Fully connected layer

    def forward(self, X):
        X = self.embedding(X)
        X, _ = self.rnn(X)
        X = self.fc(X[:, -1, :])
        return X

model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

predict = model(input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

/Users/hyungseok/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch: 0100 cost = 0.145916
Epoch: 0200 cost = 0.049905
Epoch: 0300 cost = 0.027303
Epoch: 0400 cost = 0.017529
Epoch: 0500 cost = 0.012311
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']


## LSTM Pytorch

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]

# Word processing
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

# TextLSTM Parameter
batch_size = len(sentences)
n_step = 2
n_hidden = 5

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(input)
        target_batch.append(target)
  
    return torch.LongTensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_batch(sentences)

In [12]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.embedding = nn.Embedding(n_class, n_class)  # Using embedding layer
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden, batch_first=True)
        self.fc = nn.Linear(n_hidden, n_class)  # Fully connected layer

    def forward(self, X):
        X = self.embedding(X)
        X, _ = self.lstm(X)
        X = self.fc(X[:, -1, :])
        return X

model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    optimizer.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

predict = model(input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

Epoch: 0100 cost = 0.167214
Epoch: 0200 cost = 0.059031
Epoch: 0300 cost = 0.021574
Epoch: 0400 cost = 0.011969
Epoch: 0500 cost = 0.008224
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
